In [1]:
import numpy as np
from typing import List, Tuple
import matplotlib.pyplot as plt
# ============================================================
# Data Generation (This does the same thing as the genimages.py provided)
# ============================================================
np.random.seed(0)

features = [
    [0, 0, 1, 0,
     0, 1, 1, 1,
     0, 0, 1, 0,
     0, 0, 0, 0],
    [0, 1, 0, 0,
     0, 1, 0, 0,
     0, 1, 0, 0,
     0, 1, 0, 0],
    [1, 1, 1, 1,
     0, 0, 0, 0,
     0, 0, 0, 0,
     0, 0, 0, 0],
    [1, 0, 0, 0,
     0, 1, 0, 0,
     0, 0, 1, 0,
     0, 0, 0, 1],
    [0, 0, 0, 0,
     0, 0, 0, 0,
     1, 1, 0, 0,
     1, 1, 0, 0],
    [1, 1, 1, 1,
     1, 0, 0, 1,
     1, 0, 0, 1,
     1, 1, 1, 1],
    [0, 0, 0, 0,
     0, 1, 1, 0,
     0, 1, 1, 0,
     0, 0, 0, 0],
    [0, 0, 0, 1,
     0, 0, 0, 1,
     0, 0, 0, 1,
     0, 0, 0, 1],
]

num_samples = 2000
num_features = 16
K_true = len(features)

feature_weights = 0.5 + np.random.rand(K_true, 1) * 0.5
mu_true = np.array([weight * feat for weight, feat in zip(feature_weights, features)])
latent_factors = (np.random.rand(num_samples, K_true) < 0.3).astype(float)
data = latent_factors @ mu_true + np.random.randn(num_samples, num_features)*0.1


In [2]:
from __future__ import annotations

from abc import ABC, abstractmethod
from typing import TYPE_CHECKING, List


import numpy as np


class AbstractBinaryLatentFactorApproximation(ABC):
    @property
    @abstractmethod
    def lambda_matrix(self) -> np.ndarray:
        """
        lambda_matrix: parameters variational approximation (number_of_points, number_of_latent_variables)
        """
        pass

    @abstractmethod
    def variational_expectation_step(
        self,
        x: np.ndarray,
        binary_latent_factor_model: AbstractBinaryLatentFactorModel,
    ) -> List[float]:
        pass

    @property
    def expectation_s(self):
        return self.lambda_matrix

    @property
    def expectation_ss(self):
        ess = self.lambda_matrix.T @ self.lambda_matrix
        np.fill_diagonal(ess, self.lambda_matrix.sum(axis=0))
        return ess

    @property
    def log_lambda_matrix(self) -> np.ndarray:
        return np.log(self.lambda_matrix)

    @property
    def log_one_minus_lambda_matrix(self) -> np.ndarray:
        return np.log(1 - self.lambda_matrix)

    @property
    def n(self) -> int:
        """
        Number of data points
        """
        return self.lambda_matrix.shape[0]

    @property
    def k(self) -> int:
        """
        Number of latent variables
        """
        return self.lambda_matrix.shape[1]

    def compute_free_energy(
        self,
        x: np.ndarray,
        binary_latent_factor_model: AbstractBinaryLatentFactorModel,
    ) -> float:
        """
        free energy associated with current EM parameters and data x

        :param x: data matrix (number_of_points, number_of_dimensions)
        :param binary_latent_factor_model: a binary_latent_factor_model
        :return: average free energy per data point
        """
        expectation_log_p_x_s_given_theta = (
            self._compute_expectation_log_p_x_s_given_theta(
                x, binary_latent_factor_model
            )
        )
        approximation_model_entropy = self._compute_approximation_model_entropy()
        return (
            expectation_log_p_x_s_given_theta + approximation_model_entropy
        ) / self.n

    def _compute_expectation_log_p_x_s_given_theta(
        self,
        x: np.ndarray,
        binary_latent_factor_model: AbstractBinaryLatentFactorModel,
    ) -> float:
        """
        The first term of the free energy, the expectation of log P(X,S|theta)

        :param x: data matrix (number_of_points, number_of_dimensions)
        :param binary_latent_factor_model: a binary_latent_factor_model
        :return: the expectation of log P(X,S|theta)
        """
        # (number_of_points, number_of_dimensions)
        mu_lambda = self.lambda_matrix @ binary_latent_factor_model.mu.T

        # (number_of_latent_variables, number_of_latent_variables)
        expectation_s_i_s_j_mu_i_mu_j = np.multiply(
            self.lambda_matrix.T @ self.lambda_matrix,
            binary_latent_factor_model.mu.T @ binary_latent_factor_model.mu,
        )

        expectation_log_p_x_given_s_theta = -(
            self.n * binary_latent_factor_model.d / 2
        ) * np.log(2 * np.pi * binary_latent_factor_model.variance) - (
            0.5 * binary_latent_factor_model.precision
        ) * (
            np.sum(np.multiply(x, x))
            - 2 * np.sum(np.multiply(x, mu_lambda))
            + np.sum(expectation_s_i_s_j_mu_i_mu_j)
            - np.trace(
                expectation_s_i_s_j_mu_i_mu_j
            )  # remove incorrect E[s_i s_i] = lambda_i * lambda_i
            + np.sum(  # add correct E[s_i s_i] = lambda_i
                self.lambda_matrix
                @ np.multiply(
                    binary_latent_factor_model.mu, binary_latent_factor_model.mu
                ).T
            )
        )
        expectation_log_p_s_given_theta = np.sum(
            np.multiply(
                self.lambda_matrix,
                binary_latent_factor_model.log_pi,
            )
            + np.multiply(
                1 - self.lambda_matrix,
                binary_latent_factor_model.log_one_minus_pi,
            )
        )
        return expectation_log_p_x_given_s_theta + expectation_log_p_s_given_theta

    def _compute_approximation_model_entropy(self) -> float:
        """
        Compute the model entropy

        :return: model entropy
        """
        return -np.sum(
            np.multiply(
                self.lambda_matrix,
                self.log_lambda_matrix,
            )
            + np.multiply(
                1 - self.lambda_matrix,
                self.log_one_minus_lambda_matrix,
            )
        )


In [3]:
def m_step(X, ES, ESS):
    """
    mu, sigma, pie = MStep(X,ES,ESS)

    Inputs:
    -----------------
           X: shape (N, D) data matrix
          ES: shape (N, K) E_q[s]
         ESS: shape (K, K) sum over data points of E_q[ss'] (N, K, K)
                           if E_q[ss'] is provided, the sum over N is done for you.

    Outputs:
    --------
          mu: shape (D, K) matrix of means in p(y|{s_i},mu,sigma)
       sigma: shape (,)    standard deviation in same
         pie: shape (1, K) vector of parameters specifying generative distribution for s
    """
    N, D = X.shape
    if ES.shape[0] != N:
        raise TypeError('ES must have the same number of rows as X')
    K = ES.shape[1]
    if ESS.shape == (N, K, K):
        ESS = np.sum(ESS, axis=0)
    if ESS.shape != (K, K):
        raise TypeError('ESS must be square and have the same number of columns as ES')

    mu = np.dot(np.dot(np.linalg.inv(ESS), ES.T), X).T
    sigma = np.sqrt((np.trace(np.dot(X.T, X)) + np.trace(np.dot(np.dot(mu.T, mu), ESS))
                     - 2 * np.trace(np.dot(np.dot(ES.T, X), mu))) / (N * D))
    pie = np.mean(ES, axis=0, keepdims=True)

    return mu, sigma, pie


In [4]:



class MessagePassingApproximation(AbstractBinaryLatentFactorApproximation):
    """
    bernoulli_parameter_matrix (theta): matrix of parameters bernoulli_parameter_matrix[n, i, j]
                off diagonals corresponds to \tilda{g}_{ij, \neg s_i}(s_j) for data point n
                diagonals correspond to \tilda{f}_{i}(s_i)
                (number_of_points, number_of_latent_variables, number_of_latent_variables)
    """

    def __init__(self, bernoulli_parameter_matrix: np.ndarray):
        self.bernoulli_parameter_matrix = bernoulli_parameter_matrix

    @property
    def lambda_matrix(self) -> np.ndarray:
        """
        Aggregate messages and compute parameter for Bernoulli distribution
        :return:
        """
        lambda_matrix = 1 / (1 + np.exp(-self.natural_parameter_matrix.sum(axis=1)))
        lambda_matrix[lambda_matrix == 0] = 1e-10
        lambda_matrix[lambda_matrix == 1] = 1 - 1e-10
        return lambda_matrix

    @property
    def natural_parameter_matrix(self) -> np.ndarray:
        """
        The matrix containing natural parameters (eta) of each factor
                off diagonals corresponds to \tilda{g}_{ij, \neg s_i}(s_j) for data point n
                diagonals correspond to \tilda{f}_{i}(s_i)
                (number_of_points, number_of_latent_variables, number_of_latent_variables)
        :return:
        """
        return np.log(
            np.divide(
                self.bernoulli_parameter_matrix, 1 - self.bernoulli_parameter_matrix
            )
        )

    def aggregate_incoming_binary_factor_messages(
        self, node_index: int, excluded_node_index: int
    ) -> np.ndarray:
        # (number_of_points, )
        #  exclude message from excluded_node_index -> node_index
        return (
            np.sum(
                self.natural_parameter_matrix[:, :excluded_node_index, node_index],
                axis=1,
            )
            + np.sum(
                self.natural_parameter_matrix[:, excluded_node_index + 1 :, node_index],
                axis=1,
            )
        ).reshape(
            -1,
        )

    @staticmethod
    def calculate_bernoulli_parameter(
        natural_parameter_matrix: np.ndarray,
    ) -> np.ndarray:
        bernoulli_parameter = 1 / (1 + np.exp(-natural_parameter_matrix))
        bernoulli_parameter[bernoulli_parameter == 0] = 1e-10
        bernoulli_parameter[bernoulli_parameter == 1] = 1 - 1e-10
        return bernoulli_parameter

    def variational_expectation_step(
        self, x: np.ndarray, binary_latent_factor_model: BoltzmannMachine
    ) -> List[float]:
        """
        Iteratively update singleton and binary factors
        :param x: data matrix (number_of_points, number_of_dimensions)
        :param binary_latent_factor_model: a binary_latent_factor_model
        :return: free energies after each update
        """
        free_energy = [self.compute_free_energy(x, binary_latent_factor_model)]
        for i in range(self.k):
            # singleton factor update
            natural_parameter_ii = self.calculate_singleton_message_update(
                boltzmann_machine=binary_latent_factor_model,
                x=x,
                i=i,
            )
            self.bernoulli_parameter_matrix[
                :, i, i
            ] = self.calculate_bernoulli_parameter(natural_parameter_ii)
            free_energy.append(self.compute_free_energy(x, binary_latent_factor_model))

            for j in range(i):
                # binary factor update
                natural_parameter_ij = self.calculate_binary_message_update(
                    boltzmann_machine=binary_latent_factor_model,
                    x=x,
                    i=i,
                    j=j,
                )
                self.bernoulli_parameter_matrix[
                    :, i, j
                ] = self.calculate_bernoulli_parameter(natural_parameter_ij)
                natural_parameter_ji = self.calculate_binary_message_update(
                    boltzmann_machine=binary_latent_factor_model,
                    x=x,
                    i=j,
                    j=i,
                )
                self.bernoulli_parameter_matrix[
                    :, j, i
                ] = self.calculate_bernoulli_parameter(natural_parameter_ji)
                free_energy.append(
                    self.compute_free_energy(x, binary_latent_factor_model)
                )
        return free_energy

    def calculate_binary_message_update(
        self,
        x: np.ndarray,
        boltzmann_machine: BoltzmannMachine,
        i: int,
        j: int,
    ) -> float:
        """
        Calculate new parameters for a binary factored message.

        :param x: data matrix (number_of_points, number_of_dimensions)
        :param boltzmann_machine: Boltzmann machine model
        :param i: starting node for the message
        :param j: ending node for the message
        :return: new parameter from aggregating incoming messages
        """
        natural_parameter_i_not_j = boltzmann_machine.b_index(
            x=x, node_index=i
        ) + self.aggregate_incoming_binary_factor_messages(
            node_index=i, excluded_node_index=j
        )
        w_i_j = boltzmann_machine.w_matrix_index(i, j)
        return np.log(1 + np.exp(w_i_j + natural_parameter_i_not_j)) - np.log(
            1 + np.exp(natural_parameter_i_not_j)
        )

    @staticmethod
    def calculate_singleton_message_update(
        x: np.ndarray,
        boltzmann_machine: BoltzmannMachine,
        i: int,
    ) -> float:
        """
        Calculate the parameter update for the singleton message.
        Note that this does not require any approximation.

        :param x: data matrix (number_of_points, number_of_dimensions)
        :param boltzmann_machine: Boltzmann machine model
        :param i: node to update
        :return: new parameter
        """
        return boltzmann_machine.b_index(x=x, node_index=i)


def init_message_passing(k: int, n: int) -> MessagePassingApproximation:
    """
    Message passing initialisation

    :param k: number of latent variables
    :param n: number of data points
    :return: message passing
    """
    bernoulli_parameter_matrix = np.random.random(size=(n, k, k))
    return MessagePassingApproximation(bernoulli_parameter_matrix)


In [8]:



class BoltzmannMachine(BinaryLatentFactorModel):
    def __init__(
        self,
        mu: np.ndarray,
        sigma: float,
        pi: np.ndarray,
    ):
        """
        Binary latent factor model with Boltzmann Machine terms
        """
        super().__init__(mu, sigma, pi)

    @property
    def w_matrix(self) -> np.ndarray:
        """
        Weight matrix of the Boltzmann machine

        :return: matrix of weights (number_of_latent_variables, number_of_latent_variables)
        """
        return -self.precision * (self.mu.T @ self.mu)

    def w_matrix_index(self, i, j) -> float:
        """
        Weight matrix at a specific index

        :param i: row index
        :param j: column index
        :return: weight value
        """
        return -self.precision * (self.mu[:, i] @ self.mu[:, j])

    def b(self, x) -> np.ndarray:
        """
        b term in the Boltzmann machine for all data points

        :param x: design matrix (number_of_points, number_of_dimensions)
        :return: matrix of shape (number_of_points, number_of_latent_variables)
        """
        return -(
            self.precision * x @ self.mu
            + self.log_pi_ratio
            - 0.5 * self.precision * np.multiply(self.mu, self.mu).sum(axis=0)
        )

    def b_index(self, x, node_index) -> float:
        """
        b term for a specific node in the Boltzmann machine for all data points

        :param x: design matrix (number_of_points, number_of_dimensions)
        :param node_index: node index
        :return: vector of shape (number_of_points, 1)
        """
        return -(
            self.precision * x @ self.mu[:, node_index]
            + (self.log_pi[0, node_index] - self.log_one_minus_pi[0, node_index])
            - 0.5 * self.precision * self.mu[:, node_index] @ self.mu[:, node_index]
        ).reshape(
            -1,
        )

    @property
    def log_pi_ratio(self) -> np.ndarray:
        return self.log_pi - self.log_one_minus_pi


def init_boltzmann_machine(
    x: np.ndarray,
    binary_latent_factor_approximation: AbstractBinaryLatentFactorApproximation,
) -> BinaryLatentFactorModel:
    """
    Initialise by running a maximisation step with the parameters of the binary latent factor approximation

    :param x: data matrix (number_of_points, number_of_dimensions)
    :param binary_latent_factor_approximation: a binary_latent_factor_approximation
    :return: an initialised Boltzmann machine model
    """
    mu, sigma, pi = BinaryLatentFactorModel.calculate_maximisation_parameters(
        x, binary_latent_factor_approximation
    )
    return BoltzmannMachine(mu=mu, sigma=sigma, pi=pi)


In [7]:



class BinaryLatentFactorModel(AbstractBinaryLatentFactorModel):
    def __init__(
        self,
        mu: np.ndarray,
        sigma: float,
        pi: np.ndarray,
    ):
        self._mu = mu  # (number_of_dimensions, number_of_latent_variables)
        self._sigma = sigma
        self._pi = pi  # (1, number_of_latent_variables)

    @property
    def mu(self):
        return self._mu

    @mu.setter
    def mu(self, value):
        self._mu = value

    @property
    def sigma(self):
        return self._sigma

    @sigma.setter
    def sigma(self, value):
        self._sigma = value

    @property
    def pi(self):
        return self._pi

    @pi.setter
    def pi(self, value):
        self._pi = value

    @property
    def variance(self) -> float:
        return self.sigma**2

    @staticmethod
    def calculate_maximisation_parameters(
        x: np.ndarray,
        binary_latent_factor_approximation: AbstractBinaryLatentFactorApproximation,
    ) -> Tuple[np.ndarray, float, np.ndarray]:
        return m_step(
            X=x,
            ES=binary_latent_factor_approximation.expectation_s,
            ESS=binary_latent_factor_approximation.expectation_ss,
        )

    def maximisation_step(
        self,
        x: np.ndarray,
        binary_latent_factor_approximation: AbstractBinaryLatentFactorApproximation,
    ) -> None:
        mu, sigma, pi = self.calculate_maximisation_parameters(
            x, binary_latent_factor_approximation
        )
        self.mu = mu
        self.sigma = sigma
        self.pi = pi


def init_binary_latent_factor_model(
    x: np.ndarray,
    binary_latent_factor_approximation: AbstractBinaryLatentFactorApproximation,
) -> BinaryLatentFactorModel:
    """
    Initialise by running a maximisation step with the parameters of the binary latent factor approximation

    :param x: data matrix (number_of_points, number_of_dimensions)
    :param binary_latent_factor_approximation: a binary_latent_factor_approximation
    :return: an initialised binary latent factor model
    """
    mu, sigma, pi = BinaryLatentFactorModel.calculate_maximisation_parameters(
        x, binary_latent_factor_approximation
    )
    return BinaryLatentFactorModel(mu, sigma, pi)


In [6]:
from __future__ import annotations

from abc import ABC, abstractmethod
from typing import TYPE_CHECKING

import numpy as np

if TYPE_CHECKING:
    from src.models.binary_latent_factor_approximations.abstract_binary_latent_factor_approximation import (
        AbstractBinaryLatentFactorApproximation,
    )


class AbstractBinaryLatentFactorModel(ABC):
    @property
    @abstractmethod
    def mu(self) -> np.ndarray:
        """
        matrix of means (number_of_dimensions, number_of_latent_variables)
        """
        pass

    @property
    @abstractmethod
    def variance(self) -> float:
        """
        gaussian noise parameter
        """
        pass

    @property
    @abstractmethod
    def pi(self) -> np.ndarray:
        """
        (1, number_of_latent_variables)
        """
        pass

    @abstractmethod
    def maximisation_step(
        self,
        x: np.ndarray,
        binary_latent_factor_approximation: AbstractBinaryLatentFactorApproximation,
    ) -> None:
        pass

    def mu_exclude(self, exclude_latent_index: int) -> np.ndarray:
        return np.concatenate(  # (number_of_dimensions, number_of_latent_variables-1)
            (self.mu[:, :exclude_latent_index], self.mu[:, exclude_latent_index + 1 :]),
            axis=1,
        )

    @property
    def log_pi(self) -> np.ndarray:
        return np.log(self.pi)

    @property
    def log_one_minus_pi(self) -> np.ndarray:
        return np.log(1 - self.pi)

    @property
    def precision(self) -> float:
        return 1 / self.variance

    @property
    def d(self) -> int:
        return self.mu.shape[0]

    @property
    def k(self) -> int:
        return self.mu.shape[1]


In [13]:
import matplotlib.pyplot as plt
import numpy as np



def run(x: np.ndarray, k: int, em_iterations: int, save_path: str) -> None:
    n = x.shape[0]
    message_passing = init_message_passing(k, n)
    boltzmann_machine = init_boltzmann_machine(x, message_passing)

    # pre-training features plot
    fig, axes = plt.subplots(1, k, figsize=(k * 2, 2))
    for i in range(k):
        axes[i].imshow(boltzmann_machine.mu[:, i].reshape(4, 4), cmap='gray', interpolation='none')
        axes[i].set_title(f"Feature {i+1}")
        axes[i].axis('off')
    fig.suptitle("Initial Features (Loopy BP)")
    plt.tight_layout()
    plt.savefig(save_path + "-init-latent-factors", bbox_inches="tight")
    plt.close()

    # EM
    message_passing, boltzmann_machine, free_energy = learn_binary_factors(
        x=x,
        k=k,
        em_iterations=em_iterations,
        binary_latent_factor_model=boltzmann_machine,
        binary_latent_factor_approximation=message_passing,
    )

    # post training features plot
    fig, axes = plt.subplots(1, k, figsize=(k * 2, 2))
    for i in range(k):
        axes[i].imshow(boltzmann_machine.mu[:, i].reshape(4, 4), cmap='gray', interpolation='none')
        axes[i].set_title(f"Feature {i+1}")
        axes[i].axis('off')
    fig.suptitle("Learned Features (Loopy BP)")
    plt.tight_layout()
    plt.savefig(save_path + "-latent-factors", bbox_inches="tight")
    plt.close()

    # free energy plot
    plt.title("Free Energy (Loopy BP)")
    plt.xlabel("t (EM steps)")
    plt.ylabel("Free Energy")
    plt.plot(free_energy)
    plt.savefig(save_path + "-free-energy", bbox_inches="tight")
    plt.close()


In [14]:
def is_converge(
    free_energies: List[float],
    current_lambda_matrix: np.ndarray,
    previous_lambda_matrix: np.ndarray,
    free_energy_threshold: float = 1e-6,
    lambda_threshold: float = 1e-6,
) -> bool:
    """
    Determine whether the algorithm has converged based on changes in free energy
    and the lambda matrix.

    Convergence is achieved if the change in free energy between the last two iterations
    is below a specified threshold and the change in the lambda matrix (measured by
    the Frobenius norm) is also below a specified threshold.

    Parameters
    ----------
    free_energies : List[float]
        List of free energy values recorded at each iteration.
    current_lambda_matrix : np.ndarray
        The current lambda matrix after the latest iteration.
    previous_lambda_matrix : np.ndarray
        The lambda matrix from the previous iteration.
    free_energy_threshold : float, optional
        Threshold for the change in free energy to determine convergence, by default 1e-6.
    lambda_threshold : float, optional
        Threshold for the change in the lambda matrix (Frobenius norm) to determine convergence,
        by default 1e-6.

    Returns
    -------
    bool
        True if both the change in free energy and the change in lambda matrix are below
        their respective thresholds, indicating convergence. Otherwise, False.
    """
    if len(free_energies) < 2:
        # Not enough data to determine convergence
        return False

    # Calculate the absolute change in free energy
    free_energy_change = abs(free_energies[-1] - free_energies[-2])

    # Calculate the Frobenius norm of the change in lambda matrix
    lambda_change = np.linalg.norm(current_lambda_matrix - previous_lambda_matrix)

    # Check if both changes are below their respective thresholds
    return (free_energy_change <= free_energy_threshold) and (lambda_change <= lambda_threshold)


def learn_binary_factors(
    x: np.ndarray,
    k: int,
    em_iterations: int,
    binary_latent_factor_model: 'VariationalBayes',
    binary_latent_factor_approximation: 'MeanFieldApproximation',
) -> Tuple['MeanFieldApproximation', 'VariationalBayes', List[float]]:
    """
    Perform the Expectation-Maximization (EM) algorithm to learn binary latent factors.

    This function iteratively performs the E-step and M-step to optimize the
    variational approximation of binary latent factors and update the
    variational Bayes model. It records the free energy at each iteration to
    monitor convergence.

    Parameters
    ----------
    x : np.ndarray
        Data matrix of shape (n_samples, n_dimensions), where n_samples is the
        number of data points and n_dimensions is the number of observed dimensions.
    em_iterations : int
        Maximum number of EM iterations to perform.
    binary_latent_factor_model : VariationalBayes
        An instance of VariationalBayes representing the current model.
    binary_latent_factor_approximation : MeanFieldApproximation
        An instance of MeanFieldApproximation representing the current variational
        approximation of the binary latent factors.

    Returns
    -------
    Tuple[MeanFieldApproximation, VariationalBayes, List[float]]
        A tuple containing:
        - The updated MeanFieldApproximation instance.
        - The updated VariationalBayes model.
        - A list of free energy values recorded at each EM iteration.
    """
    # Initialize the list of free energies with the initial free energy
    free_energies: List[float] = [
        binary_latent_factor_approximation.compute_free_energy(
            x, binary_latent_factor_model
        )
    ]

    for iteration in range(1, em_iterations + 1):
        # Store the previous lambda matrix for convergence checking
        previous_lambda_matrix = np.copy(binary_latent_factor_approximation.lambda_matrix)

        # E-step: Update the variational approximation (lambda matrix)
        free_energy_history = binary_latent_factor_approximation.variational_expectation_step(
            x=x,
            binary_latent_factor_model=binary_latent_factor_model,
        )

        # M-step: Update the variational Bayes model parameters
        binary_latent_factor_model.maximisation_step(
            x=x,
            binary_latent_factor_approximation=binary_latent_factor_approximation,
        )

        # Compute and record the new free energy
        current_free_energy = binary_latent_factor_approximation.compute_free_energy(
            x, binary_latent_factor_model
        )
        free_energies.append(current_free_energy)

        # Check for convergence
        if is_converge(
            free_energies=free_energies,
            current_lambda_matrix=binary_latent_factor_approximation.lambda_matrix,
            previous_lambda_matrix=previous_lambda_matrix,
        ):
            print(f"current K = {k},"
                  f" Convergence achieved at iteration {iteration},"
                  f" Free Energy at Convergence: {current_free_energy}.")
            break


    return binary_latent_factor_approximation, binary_latent_factor_model, free_energies

In [15]:
import os
from dataclasses import asdict

import jax
import jax.numpy as jnp
import numpy as np
import pandas as pd

# Constants for output directories and random seed
OUTPUTS_FOLDER = "LoopyBP"
DEFAULT_SEED = 43

if __name__ == "__main__":
    np.random.seed(DEFAULT_SEED)

    if not os.path.exists(OUTPUTS_FOLDER):
        os.makedirs(OUTPUTS_FOLDER)


    number_of_images = 2000
    x = data
    k = 8
    em_iterations = 100
    e_maximum_steps = 50
    e_convergence_criterion = 0


    Q6_OUTPUT_FOLDER = os.path.join(OUTPUTS_FOLDER, "q6")
    if not os.path.exists(Q6_OUTPUT_FOLDER):
        os.makedirs(Q6_OUTPUT_FOLDER)
    run(x, k, em_iterations, save_path=os.path.join(Q6_OUTPUT_FOLDER, "all"))

/var/folders/18/wz5v3z6x7_gdgjx4t9zvjzxc0000gn/T/ipykernel_52814/3139600142.py:134: RuntimeWarning: overflow encountered in exp
  return np.log(1 + np.exp(w_i_j + natural_parameter_i_not_j)) - np.log(
